## **FakeNews Detection Using BERT | bert-base-uncased pretrained model**

### **Dataset size**: Trained the model on FNC-1 with 64,000 headlines and articles during initial training process and again trained the model on another set of FNC-1 with 49,998 headlines and articles

1. Trained the FNC1 dataset on 4 class labels
2. Validated and tested the FNC1 dataset on 4 class labels
3. Trained the FNC1 dataset on 3 class labels
4. Tested the FNC1 dataset on 3 class labels
5. Evaluation metric: Accuracy , and f1 score

## **Libraries**

1. Pytorch
2. SimpleTransformers
3. Transformers
4. spacy_transformers
5. spacy
6. Classification Models
7. BertModel and BertPretrainedModel
6. sklearn (evaluation metric)

## **Runtime type: GPU-cuda**

## **Load the required libraries**

In [ ]:
!pip install transformers==4.2.0


     |████████████████████████████████| 1.8MB 14.6MB/s 
     |████████████████████████████████| 901kB 47.3MB/s 
     |████████████████████████████████| 2.9MB 61.9MB/s 


In [ ]:
!pip install simpletransformers

In [ ]:
import os
import csv
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split

def fnc(path_headlines, path_bodies):   # create function to merge train_bodies and test_bodies with stance (target variable) using BodyID as comman variable


    map = {'agree': 0, 'disagree':1, 'discuss':2, 'unrelated':3}

    with open(path_bodies, encoding='utf_8') as fb:  # Body ID,articleBody
        body_dict = {}
        lines_b = csv.reader(fb)
        for i, line in enumerate(tqdm(list(lines_b), ncols=80, leave=False)):
            if i > 0:
                body_id = int(line[0].strip())
                body_dict[body_id] = line[1]

    with open(path_headlines, encoding='utf_8') as fh: # Headline,Body ID,Stance
        lines_h = csv.reader(fh)
        h = []
        b = []
        l = []
        for i, line in enumerate(tqdm(list(lines_h), ncols=80, leave=False)):
            if i > 0:
                body_id = int(line[1].strip())
                label = line[2].strip()
                if label in map and body_id in body_dict:
                    h.append(line[0])
                    l.append(map[line[2]])
                    b.append(body_dict[body_id])
    return h, b, l

data_dir = '/content'
headlines, bodies, labels = fnc(
    os.path.join(data_dir, 'combined_stances_train.csv'),
    os.path.join(data_dir, 'combined_bodies_train.csv')
)

list_of_tuples = list(zip(headlines, bodies, labels))
df = pd.DataFrame(list_of_tuples, columns=['text_a', 'text_b', 'labels'])
train_df, val_df = train_test_split(df)
labels_val = pd.Series(val_df['labels']).to_numpy()

headlines, bodies, labels = fnc(
    os.path.join(data_dir, 'combined_stances_test.csv'),
    os.path.join(data_dir, 'combined_bodies_test.csv')
)

list_of_tuples = list(zip(headlines, bodies, labels))
test_df = pd.DataFrame(list_of_tuples, columns=['text_a', 'text_b', 'labels'])
labels_test = pd.Series(test_df['labels']).to_numpy()

In [ ]:
! pip install spacy_transformers

In [ ]:
!pip install spacy

In [ ]:
!pip install thinc  # not required


!pip install thinc


In [ ]:
!pip install transformers==4.2.0

In [ ]:
pip install wandb

In [ ]:
# importing the libraries for Classification task

from simpletransformers.classification import ClassificationModel, ClassificationArgs


In [ ]:


from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel

from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # pre-trained model
model = AutoModel.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# fine-tune the model with specific parameters designed for classification task, initially we train the model for 3-class labels

Bertmodel = ClassificationModel('bert', 'bert-base-uncased',use_cuda=True, num_labels=3, args={
    'learning_rate':3e-5,
    'num_train_epochs': 5,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'process_count': 10,
    'train_batch_size': 4,
    'eval_batch_size': 4,
    'max_seq_length': 512,
    'fp16': True,
    
})

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
train_rel=train_df[train_df['labels']!=3] # exclude unrelated labels

In [ ]:
Bertmodel_3class.train_model(train_rel)

  0%|          | 0/12779 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/3195 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/3195 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/3195 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/3195 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/3195 [00:00<?, ?it/s]

(15975, 0.31541862647992486)

In [ ]:
train_rel.shape

(12780, 3)

In [ ]:
val_df.shape

(16052, 3)

In [ ]:
test=test_df[test_df['labels']!=3]

In [ ]:
import numpy as np
_, model_outputs_train, _ = Bertmodel.eval_model(train_rel)  # evaluate the train data

preds_train = np.argmax(model_outputs_train, axis=1)  # predict train labels and select target label which have maximum probability in a numpy array

  0%|          | 0/12780 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3195 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

fnclabels_train_3l = pd.Series(train_rel['labels']).to_numpy() # convert thr train target labels to numpy array
eval_report_train_3l = classification_report(fnclabels_train_3l, preds_train)
print(eval_report_train_3l)

confusion_matrix(preds_train,fnclabels_train_3l)



              precision    recall  f1-score   support

           0       0.98      0.97      0.98      3703
           1       0.95      0.97      0.96      1677
           2       0.99      0.99      0.99      7400

    accuracy                           0.98     12780
   macro avg       0.97      0.98      0.98     12780
weighted avg       0.98      0.98      0.98     12780



array([[3597,   43,   23],
       [  65, 1623,   24],
       [  41,   11, 7353]])

In [ ]:
import numpy as np
val=val_df[val_df['labels']!=3]
_, model_outputs_val, _ = Bertmodel.eval_model(val)  # evaluate validation data

preds_val= np.argmax(model_outputs_val, axis=1) # predict validation labels and select target label which have maximum probability in a numpy array

  0%|          | 0/4210 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1053 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

fnclabels_val_3l = pd.Series(val['labels']).to_numpy() # convert thr validation target labels to numpy array
eval_report_val_3l = classification_report(fnclabels_val_3l, preds_val)
print(eval_report_val_3l)

confusion_matrix(preds_val,fnclabels_val_3l)

              precision    recall  f1-score   support

           0       0.83      0.85      0.84      1232
           1       0.68      0.69      0.69       565
           2       0.94      0.93      0.94      2413

    accuracy                           0.88      4210
   macro avg       0.82      0.82      0.82      4210
weighted avg       0.88      0.88      0.88      4210



array([[1045,  115,   94],
       [ 114,  391,   68],
       [  73,   59, 2251]])

In [ ]:
train_rel['labels'].value_counts()

2    7400
0    3703
1    1677
Name: labels, dtype: int64

In [ ]:
val['labels'].value_counts()

2    2413
0    1232
1     565
Name: labels, dtype: int64

In [ ]:
test['labels'].value_counts()

2    4643
0    2237
1    1069
Name: labels, dtype: int64

## **Evaluation with test data**

In [ ]:
import numpy as np
_, model_outputs_test, _ = Bertmodel.eval_model(test) # evaluate test data

preds_test = np.argmax(model_outputs_test, axis=1)  # predict test labels and select target label which have maximum probability in a numpy array

  0%|          | 0/7949 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1988 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

fnclabels_test_3l = pd.Series(test['labels']).to_numpy()  # convert thr test target labels to numpy array
eval_report_test_3l = classification_report(fnclabels_test_3l, preds_test)
print(eval_report_test_3l)

              precision    recall  f1-score   support

           0       0.63      0.69      0.66      2237
           1       0.55      0.40      0.46      1069
           2       0.84      0.86      0.85      4643

    accuracy                           0.75      7949
   macro avg       0.67      0.65      0.66      7949
weighted avg       0.74      0.75      0.74      7949



In [ ]:
confusion_matrix(preds_test,fnclabels_test_3l)


array([[1534,  413,  479],
       [ 175,  432,  184],
       [ 528,  224, 3980]])

In [ ]:
import pickle
filename1 = open('finalized_BERT.pkl','wb')
pickle.dump(Bertmodel,filename1)
filename1.close()

# **FNC1**

In [ ]:
import pandas as pd
fnc=pd.read_csv('/content/Train_data2_combined.csv')
fnc_=fnc[['Headline','articleBody','stance_cat']]

In [ ]:
fnc_.shape

(49972, 3)

In [ ]:
fnc_=fnc_[fnc_['stance_cat']!=3]

In [ ]:
fnc_.head(3)

,Headline,articleBody,stance_cat
1,hundreds palestinians flee floods gaza israel ...,hundreds palestinians evacuated homes sunday m...,0
4,spider burrowed tourist stomach chest,fear arachnophobes story bunbury spiderman mig...,1
5,nasa confirms earth experience days total dark...,thousands people duped fake news story claimin...,0


In [ ]:
fnc_['stance_cat'].unique()

array([0, 1, 2])

In [ ]:
fnc_=fnc_.rename({"Headline":'text_a',"articleBody":'text_b','stance_cat':'labels'},axis=1)
fnc_.head()

,text_a,text_b,labels
1,hundreds palestinians flee floods gaza israel ...,hundreds palestinians evacuated homes sunday m...,0
4,spider burrowed tourist stomach chest,fear arachnophobes story bunbury spiderman mig...,1
5,nasa confirms earth experience days total dark...,thousands people duped fake news story claimin...,0
8,banksy arrested real identity revealed hoax la...,seen story floating around facebook feed banks...,0
10,gateway pundit,british rapper whose father awaiting trial man...,2


In [ ]:
fnc.shape

(49972, 12)

In [ ]:
fnc_.shape

(13427, 3)

In [ ]:
from sklearn.model_selection import train_test_split

train_fnc, val_fnc = train_test_split(fnc_)


In [ ]:
train_fnc.shape

(10070, 3)

In [ ]:
val_fnc.shape

(3357, 3)

In [ ]:
model_ = ClassificationModel('bert', 'bert-base-uncased',use_cuda=True, num_labels=3, args={
    'learning_rate':3e-5,
    'num_train_epochs': 5,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'process_count': 10,
    'train_batch_size': 4,
    'eval_batch_size': 4,
    'max_seq_length': 512,
    'fp16': True,
    
})

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
Bertmodel_3class.train_model(train_fnc)


  0%|          | 0/10070 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/2518 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/2518 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/2518 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/2518 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/2518 [00:00<?, ?it/s]

(12590, 0.2659494412461167)

In [ ]:
import pickle
fncBertmodel3class = open('fncBERTmodel_3class.sav','wb')
pickle.dump(Bertmodel_3class,fncBertmodel3class)
fncBertmodel3class.close()

fncBertmodel3class_ = open('fncBERTmodel_3class.pkl','wb')
pickle.dump(Bertmodel_3class,fncBertmodel3class_)
fncBertmodel3class_.close()

In [ ]:
_, fncmodel_outputs_train3labels, _ = Bertmodel_3class.eval_model(train_fnc)
import numpy as np
fncpreds_train_3labels= np.argmax(fncmodel_outputs_train3labels, axis=1)
fnclabels_train_3l = pd.Series(train_fnc['labels']).to_numpy()

  0%|          | 0/10070 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/2518 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report

eval_report_fnctrain_3l = classification_report(fnclabels_train_3l, fncpreds_train_3labels)
print('fnc train 3 labels report', eval_report_fnctrain_3l)

fnc train 3 labels report               precision    recall  f1-score   support

           0       0.99      0.99      0.99      2745
           1       0.97      0.97      0.97       617
           2       1.00      1.00      1.00      6708

    accuracy                           1.00     10070
   macro avg       0.99      0.99      0.99     10070
weighted avg       1.00      1.00      1.00     10070



In [ ]:

from sklearn.metrics import confusion_matrix

confusion_matrix(fnclabels_train_3l, fncpreds_train_3labels) # using sklearn

array([[2727,   15,    3],
       [  17,  597,    3],
       [   7,    1, 6700]])

In [ ]:
_, fncmodel_outputs_val3labels, _ = Bertmodel_3class.eval_model(val_fnc)
fncpreds_val_3labels= np.argmax(fncmodel_outputs_val3labels, axis=1)
fnclabels_val_3l = pd.Series(val_fnc['labels']).to_numpy()

  0%|          | 0/3357 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/840 [00:00<?, ?it/s]

In [ ]:
fncpreds_val_3labels

array([2, 2, 0, ..., 2, 2, 2])

In [ ]:
calculate_f1_scores_3labels(fncpreds_val_3labels,fnclabels_val_3l)


'F1 macro: 92.863% \nF1 agree: 94.143% \nF1 disagree: 86.353% \nF1 discuss: 98.094% \n'

In [ ]:
eval_report_fncval_3l = classification_report(fnclabels_val_3l, fncpreds_val_3labels)
print('validation 3 labels report', eval_report_fncval_3l)

validation 3 labels report               precision    recall  f1-score   support

           0       0.94      0.94      0.94       933
           1       0.86      0.87      0.86       223
           2       0.98      0.98      0.98      2201

    accuracy                           0.96      3357
   macro avg       0.93      0.93      0.93      3357
weighted avg       0.96      0.96      0.96      3357



In [ ]:
confusion_matrix(fnclabels_val_3l, fncpreds_val_3labels) # using sklearn

array([[ 876,   21,   36],
       [  22,  193,    8],
       [  30,   10, 2161]])

In [ ]:
fncpreds_val_3labels

array([2, 2, 0, ..., 2, 2, 2])

## **Predictions on Test data | Sentence pair classification**

In [ ]:
# example

predictions, raw_outputs = Bertmodel_3class.predict(
    [
        [
            "hundreds palestinians flee floods gaza israel opens dams",
            "hundreds palestinians evacuated homes sunday morning israeli authorities opened number dams near border flooding gaza valley wake recent severe winter storm gaza ministry interior said statement civil defense services teams ministry public works evacuated families sides gaza valley wadi gaza homes flooded water levels reached three meters gaza experienced flooding recent days amid major storm saw temperatures drop frigid rain pour storm displaced dozens caused hardship tens thousands including many approximately palestinians left homeless israel assault summer suffering compounded fact israel maintained complete siege gaza last eight years severely limiting electricity availability fuel generators also prevented displaced rebuilding homes construction materials largely banned entering gaza civil defense services spokesman muhammad almidana warned harm could caused israel opens dams area noting water currently flowing high speed israel border valley mediterranean sea evacuated families sent shelters sponsored unrwa un agency palestinian refugees albureij refugee camp alzahra neighborhood central gaza strip gaza valley wadi gaza wetland located central gaza strip alnuseirat refugee camp almoghraqa called habesor hebrew flows two streams one whose source runs near beersheba near hebron israeli dams river collect rainwater dried wetlands inside gaza destroyed source surface water area locals continued use dispose waste lack ways however creating environmental hazard first time israeli authorities opened gaza valley dams dec israeli authorities also opened dams amid heavy flooding gaza strip resulting floods damaged dozens homes forces many families area homes dams opened well forcing families homes time civil defense services said managed save seven people risk drowning",
        ]
    ]
)

print(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

## **Load test data**

In [ ]:
fnc_1=pd.read_csv('/semantic_Test_Data.csv')
fnc_1=fnc_1[['Headline','articleBody']]

In [ ]:
fnc_1=fnc_1.rename({"Headline":'text_a',"articleBody":'text_b'},axis=1)
fnc_1.head()

,text_a,text_b
0,ferguson riots pregnant woman loses eye cops f...,respected senior french police officer investi...
1,crazy conservatives sure gitmo detainee killed...,dave morin social networking company path repo...
2,russian guy says justin bieber ringtone saved ...,bereaved afghan mother took revenge taliban wa...
3,zombie cat buried kitty believed dead meows ba...,hewlettpackard officially splitting two follow...
4,argentina president adopts boy end werewolf curse,airline passenger headed dallas removed plane ...


In [ ]:
fnc_res=fnc_[['text_a','text_b','labels']].iloc[0:20]

In [ ]:
#example 2
predictions,raw_outputs=Bertmodel_3class.predict([[fnc_2['text_a'].iloc[2],fnc_2['text_b'].iloc[2]]])
predictions

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([0])

In [ ]:
fnc_withoutLabels=fnc_[['text_a','text_b']]

len(fnc_withoutLabels)

13427

In [ ]:
predicted_labels_res=[]
for i in range(0,10):
     predictions,raw_outputs=Bertmodel_3class.predict([[fnc_2['text_a'].iloc[i],fnc_2['text_b'].iloc[i]]])
     predicted_labels_res.append(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
predicted_labels_res

[array([0]),
 array([1]),
 array([0]),
 array([0]),
 array([2]),
 array([0]),
 array([2]),
 array([2]),
 array([0]),
 array([2])]

In [ ]:
tested_labels=pd.DataFrame(predicted_labels_res,columns=['tested_labels_1'])
fnc_res=fnc_.reset_index(drop=True)
tested_labels

,tested_labels_1
0,0
1,1
2,0
3,0
4,2
5,0
6,2
7,2
8,0
9,2


In [ ]:
fnc_res['labels_tested']=tested_labels
fnc_res[['text_a','text_b','labels','labels_tested']].head(10)

,text_a,text_b,labels,labels_tested
0,hundreds palestinians flee floods gaza israel ...,hundreds palestinians evacuated homes sunday m...,0,0.0
1,spider burrowed tourist stomach chest,fear arachnophobes story bunbury spiderman mig...,1,1.0
2,nasa confirms earth experience days total dark...,thousands people duped fake news story claimin...,0,0.0
3,banksy arrested real identity revealed hoax la...,seen story floating around facebook feed banks...,0,0.0
4,gateway pundit,british rapper whose father awaiting trial man...,2,2.0
5,woman detained lebanon albaghdadi wife iraq says,iraqi official denied woman detained lebanon w...,0,0.0
6,soon marijuana may lead ticket arrest new york,campaigning promise reform stopandfrisk mayor ...,2,2.0
7,boko haram denies nigeria ceasefire claim,abuja nigeria leader nigeria islamist extremis...,2,2.0
8,robert plant rip million contract,led zeppelin fans disappointed learn reunion t...,0,0.0
9,isil beheads american photojournalist iraq,james foley american journalist went missing s...,2,2.0


## Evaluation | FNC test data with labels

In [ ]:
from scipy.stats.stats import pearsonr,spearmanr

result, model_outputs, wrong_predictions = Bertmodel_3class.eval_model(val_fnc, pearson_corr=pearsonr, spearman_corr=spearmanr)


  0%|          | 0/3357 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/840 [00:00<?, ?it/s]

In [ ]:
result

{'eval_loss': 0.2702985772634141,
 'mcc': 0.9224942283668013,
 'pearson_corr': (0.9387112272299849, 0.0),
 'spearman_corr': SpearmanrResult(correlation=0.942484599966097, pvalue=0.0)}

In [ ]:
predicted_val_fnc= np.argmax(model_outputs, axis=1)
predicted_val_fnc

array([2, 2, 0, ..., 2, 2, 2])


# **Evaluating the Model**


This returns three values.

**result**: The evaluation result in the form of a dict . This will include any metric functions passed into the eval_model() method.

**model_outputs**: A list of model outputs for each item in the evaluation dataset.

**wrong_predictions**: A list of InputFeature of each incorrect prediction. The text may be obtained from the InputFeature.text_a attribute. May not be particularly useful in regression tasks. (The InputFeature class can be found in the utils.py file in the repo )

With the hyperparameter values specified earlier, We obtained the following (very satisfactory) results.

pearson_corr = 0.938
spearman_corr = 0.942